# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType

from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler, \
    RegexTokenizer, CountVectorizer,IDF, StringIndexer, PCA, StopWordsRemover
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import re

In [2]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
stack_overflow_data = '../spark_project_data/mini_sparkify_event_data.json'
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [4]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [5]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
df.describe().show()

+-------+------------------+----------+---------+------+------------------+--------+-----------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+-----------------+
|summary|            artist|      auth|firstName|gender|     itemInSession|lastName|           length| level|         location|method|   page|        registration|        sessionId|                song|            status|                  ts|           userAgent|           userId|
+-------+------------------+----------+---------+------+------------------+--------+-----------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+-----------------+
|  count|            228108|    286500|   278154|278154|            286500|  278154|           228108|286500|           278154|286500| 286500|            

In [7]:
col='userId'
df.select(col).distinct().sort(col).collect()

[Row(userId=''),
 Row(userId='10'),
 Row(userId='100'),
 Row(userId='100001'),
 Row(userId='100002'),
 Row(userId='100003'),
 Row(userId='100004'),
 Row(userId='100005'),
 Row(userId='100006'),
 Row(userId='100007'),
 Row(userId='100008'),
 Row(userId='100009'),
 Row(userId='100010'),
 Row(userId='100011'),
 Row(userId='100012'),
 Row(userId='100013'),
 Row(userId='100014'),
 Row(userId='100015'),
 Row(userId='100016'),
 Row(userId='100017'),
 Row(userId='100018'),
 Row(userId='100019'),
 Row(userId='100021'),
 Row(userId='100022'),
 Row(userId='100023'),
 Row(userId='100024'),
 Row(userId='100025'),
 Row(userId='101'),
 Row(userId='102'),
 Row(userId='103'),
 Row(userId='104'),
 Row(userId='105'),
 Row(userId='106'),
 Row(userId='107'),
 Row(userId='108'),
 Row(userId='109'),
 Row(userId='11'),
 Row(userId='110'),
 Row(userId='111'),
 Row(userId='112'),
 Row(userId='113'),
 Row(userId='114'),
 Row(userId='115'),
 Row(userId='116'),
 Row(userId='117'),
 Row(userId='118'),
 Row(userId='

In [8]:
df = df.withColumn("userId", df["userId"].cast("bigint"))
df = df.withColumn("status", df["status"].cast("string"))

In [9]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: string (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: long (nullable = true)



In [10]:
df.count()

286500

In [11]:
def show_value_counts(dataframe, unique_value_filter = 30):
    """
    Prints out null_values_counts and distinct_values_counts of the dataframe
    
    Input: df - a spark dataframe
           unique_value_filter(int) - number of unique value to be filtered
    
    Output:
            print(column_name)
            print(null_value_counts and %) 
            print(distinct_value_counts)
            print(distinct_values) if less than 100
            
    """
    
    color_counts = lambda x: '\033[1m'+'\033[91m'+str(x)+'\033[0m' if x>0 else '\033[1m'+'\033[92m'+str(x)+'\033[0m'
    
    for col in df.columns:
        print('\033[1m'+'\033[95m'+col+'\033[0m','-'*(100-len(col)))
        null_value_counts = df.filter(df[col].isNull()).count()
        distinct_value_counts = df.select(col).distinct().count()
        print('Null Value Count: {} / {}%'.format(color_counts(null_value_counts),round(null_value_counts/df.count()*100,2)))
        print('Unique Value Count:', distinct_value_counts)
        if distinct_value_counts < unique_value_filter+1:
            df.select(col).distinct().sort(col).show()
        print()

In [12]:
show_value_counts(df)

artist ----------------------------------------------------------------------------------------------
Null Value Count: 58392 / 20.38%
Unique Value Count: 17656

auth ------------------------------------------------------------------------------------------------
Null Value Count: 0 / 0.0%
Unique Value Count: 4
+----------+
|      auth|
+----------+
| Cancelled|
|     Guest|
| Logged In|
|Logged Out|
+----------+


firstName -------------------------------------------------------------------------------------------
Null Value Count: 8346 / 2.91%
Unique Value Count: 190

gender ----------------------------------------------------------------------------------------------
Null Value Count: 8346 / 2.91%
Unique Value Count: 3
+------+
|gender|
+------+
|  null|
|     F|
|     M|
+------+


itemInSession ---------------------------------------------------------------------------------------
Null Value Count: 0 / 0.0%
Unique Value Count: 1322

lastName ---------------------------------------

In [116]:
# display userAgent
col='userAgent'
distinct_values = df.select(col).distinct().sort(col).collect()
for i in distinct_values:
    print(i)

Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.3 (KHTML, like Gecko) Version/8.0 Safari/600.1.3"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.8 (KHTML, like Gecko) Version/8.0 Safari/600.1.8"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/6.1.5 Safari/537.77.4"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
Row(userAgent='"Mozilla/5.0 (Macintosh; 

In [14]:
# investigate on those 8346 rows without userId
# save the rows to a Pandas DataFrame
pd_df = df.filter((df.userId.isNull())).toPandas()

In [15]:
pd_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,100,None,None,free,None,GET,Home,None,8,None,200,1538355745000,None,None
1,None,Logged Out,None,None,101,None,None,free,None,GET,Help,None,8,None,200,1538355807000,None,None
2,None,Logged Out,None,None,102,None,None,free,None,GET,Home,None,8,None,200,1538355841000,None,None
3,None,Logged Out,None,None,103,None,None,free,None,PUT,Login,None,8,None,307,1538355842000,None,None
4,None,Logged Out,None,None,2,None,None,free,None,GET,Home,None,240,None,200,1538356678000,None,None


In [16]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8346 entries, 0 to 8345
Data columns (total 18 columns):
artist           0 non-null object
auth             8346 non-null object
firstName        0 non-null object
gender           0 non-null object
itemInSession    8346 non-null int64
lastName         0 non-null object
length           0 non-null object
level            8346 non-null object
location         0 non-null object
method           8346 non-null object
page             8346 non-null object
registration     0 non-null object
sessionId        8346 non-null int64
song             0 non-null object
status           8346 non-null object
ts               8346 non-null int64
userAgent        0 non-null object
userId           0 non-null object
dtypes: int64(3), object(15)
memory usage: 1.1+ MB


In [17]:
pd_df.describe()

,itemInSession,sessionId,ts
count,8346.000000,8346.000000,8.346000e+03
mean,98.251258,1007.029954,1.540889e+12
std,125.863046,735.100827,1.529748e+09
min,0.000000,1.000000,1.538356e+12
25%,15.000000,305.000000,1.539607e+12
50%,54.000000,919.000000,1.540843e+12
75%,133.000000,1646.000000,1.542213e+12
max,1288.000000,2474.000000,1.543622e+12


In [18]:
# show categorical feature value counts of those 8346 rows
non_null_cat_cols = ['auth','level','method','page','status']

for col in non_null_cat_cols:
    print(col)
    print(pd_df[col].value_counts())
    print()

auth
Logged Out    8249
Guest           97
Name: auth, dtype: int64

level
paid    5729
free    2617
Name: level, dtype: int64

method
GET    5100
PUT    3246
Name: method, dtype: int64

page
Home                   4375
Login                  3241
About                   429
Help                    272
Register                 18
Error                     6
Submit Registration       5
Name: page, dtype: int64

status
200    5094
307    3246
404       6
Name: status, dtype: int64



In [19]:
# check row counts and the diff
df.count(),pd_df.shape[0],df.count()-pd_df.shape[0]

(286500, 8346, 278154)

In [20]:
# base on the EDA we can drop those 8346 rows, because those rows have nothing to do with Churn prediction
df = df.dropna(how = "any", subset = ['userId','sessionId'])

In [21]:
df.count()

278154

In [22]:
show_value_counts(df)

artist ----------------------------------------------------------------------------------------------
Null Value Count: 50046 / 17.99%
Unique Value Count: 17656

auth ------------------------------------------------------------------------------------------------
Null Value Count: 0 / 0.0%
Unique Value Count: 2
+---------+
|     auth|
+---------+
|Cancelled|
|Logged In|
+---------+


firstName -------------------------------------------------------------------------------------------
Null Value Count: 0 / 0.0%
Unique Value Count: 189

gender ----------------------------------------------------------------------------------------------
Null Value Count: 0 / 0.0%
Unique Value Count: 2
+------+
|gender|
+------+
|     F|
|     M|
+------+


itemInSession ---------------------------------------------------------------------------------------
Null Value Count: 0 / 0.0%
Unique Value Count: 1311

lastName ----------------------------------------------------------------------------------------

In [23]:
# investigate on those 50046 rows without song
# save the rows to a Pandas DataFrame
pd_df_2 = df.filter((df.song.isNull())).toPandas()

In [24]:
pd_df_2.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Colin,M,54,Freeman,None,paid,"Bakersfield, CA",PUT,Add to Playlist,1538173362000,29,None,200,1538352905000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,None,Logged In,Micah,M,84,Long,None,free,"Boston-Cambridge-Newton, MA-NH",GET,Roll Advert,1538331630000,8,None,200,1538353150000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,None,Logged In,Micah,M,86,Long,None,free,"Boston-Cambridge-Newton, MA-NH",PUT,Thumbs Up,1538331630000,8,None,307,1538353376000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
3,None,Logged In,Alexi,F,4,Warren,None,paid,"Spokane-Spokane Valley, WA",GET,Downgrade,1532482662000,53,None,200,1538354749000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,54
4,None,Logged In,Alexi,F,7,Warren,None,paid,"Spokane-Spokane Valley, WA",PUT,Thumbs Up,1532482662000,53,None,307,1538355255000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,54


In [25]:
count_cols = ['auth', 'gender', 'level', 'method', 'page', 'status']

for col in count_cols:
    print(col)
    print(pd_df_2[col].value_counts())
    print()

auth
Logged In    49994
Cancelled       52
Name: auth, dtype: int64

gender
F    27882
M    22164
Name: gender, dtype: int64

level
paid    36316
free    13730
Name: level, dtype: int64

method
PUT    29710
GET    20336
Name: method, dtype: int64

page
Thumbs Up                    12551
Home                         10082
Add to Playlist               6526
Add Friend                    4277
Roll Advert                   3933
Logout                        3226
Thumbs Down                   2546
Downgrade                     2055
Settings                      1514
Help                          1454
Upgrade                        499
About                          495
Save Settings                  310
Error                          252
Submit Upgrade                 159
Submit Downgrade                63
Cancellation Confirmation       52
Cancel                          52
Name: page, dtype: int64

status
200    26610
307    23184
404      252
Name: status, dtype: int64



In [26]:
# is free level relevant?
df.groupBy('level').count().show()

+-----+------+
|level| count|
+-----+------+
| free| 55721|
| paid|222433|
+-----+------+



In [39]:
pd_df.registration.sum()- pd_df.ts.sum()

-12860262767667000

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [41]:
df.groupBy('page').count().sort(desc('count')).show()

+--------------------+------+
|                page| count|
+--------------------+------+
|            NextSong|228108|
|           Thumbs Up| 12551|
|                Home| 10082|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|         Roll Advert|  3933|
|              Logout|  3226|
|         Thumbs Down|  2546|
|           Downgrade|  2055|
|            Settings|  1514|
|                Help|  1454|
|             Upgrade|   499|
|               About|   495|
|       Save Settings|   310|
|               Error|   252|
|      Submit Upgrade|   159|
|    Submit Downgrade|    63|
|              Cancel|    52|
|Cancellation Conf...|    52|
+--------------------+------+



In [86]:
df.filter("page = 'Cancellation Confirmation'").select('userId').sort('userId').collect()

[Row(userId=3),
 Row(userId=12),
 Row(userId=17),
 Row(userId=18),
 Row(userId=28),
 Row(userId=29),
 Row(userId=32),
 Row(userId=51),
 Row(userId=53),
 Row(userId=54),
 Row(userId=58),
 Row(userId=70),
 Row(userId=73),
 Row(userId=87),
 Row(userId=101),
 Row(userId=103),
 Row(userId=105),
 Row(userId=106),
 Row(userId=121),
 Row(userId=122),
 Row(userId=125),
 Row(userId=129),
 Row(userId=143),
 Row(userId=100001),
 Row(userId=100003),
 Row(userId=100005),
 Row(userId=100006),
 Row(userId=100007),
 Row(userId=100009),
 Row(userId=100011),
 Row(userId=100012),
 Row(userId=100013),
 Row(userId=100014),
 Row(userId=100015),
 Row(userId=100017),
 Row(userId=100019),
 Row(userId=100021),
 Row(userId=100022),
 Row(userId=100023),
 Row(userId=100024),
 Row(userId=100025),
 Row(userId=200001),
 Row(userId=200011),
 Row(userId=200015),
 Row(userId=200016),
 Row(userId=200017),
 Row(userId=200018),
 Row(userId=200020),
 Row(userId=200021),
 Row(userId=200024),
 Row(userId=300001),
 Row(userId=3

In [42]:
pd_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [88]:
# sample logs for FE
df.select(['auth', 'gender', 'itemInSession',
       'length', 'level', 'method', 'page', 'registration','ts',
       'sessionId', 'status']).where(df.userId == 22).show(100)

+---------+------+-------------+---------+-----+------+-----------+-------------+-------------+---------+------+
|     auth|gender|itemInSession|   length|level|method|       page| registration|           ts|sessionId|status|
+---------+------+-------------+---------+-----+------+-----------+-------------+-------------+---------+------+
|Logged In|     F|            0|     null| free|   GET|       Home|1537681013000|1541720842000|      179|   200|
|Logged In|     F|            1|     null| free|   PUT| Add Friend|1537681013000|1541720843000|      179|   307|
|Logged In|     F|            2|     null| free|   PUT| Add Friend|1537681013000|1541720844000|      179|   307|
|Logged In|     F|            3|259.76118| free|   PUT|   NextSong|1537681013000|1541720868000|      179|   200|
|Logged In|     F|            4|655.77751| free|   PUT|   NextSong|1537681013000|1541721127000|      179|   200|
|Logged In|     F|            0|     null| free|   GET|       Home|1537681013000|1542987594000| 

In [92]:
# sample logs for FE
df.select(['song']).where(df.userId == 3).distinct().count()

212

In [114]:
c = df.filter(df.userId == 3).groupBy('song').count().toPandas()
c[c['count']>1]

,song,count
53,None,40
67,You're The One,3
157,Siguiendo La Luna,2


# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# FE Note:

In [ ]:
# Features : ——————————————————
    
# key_userId

# (drop page == ‘cancelations’x2)
# ts to yyyy/mm/dd/hh

# X cat_most_listened_artist, most_linstend_song??? - can be non-singular value

# cat_top_20_artist_lover? 
# cat_top_50_artist_lover? 
# cat_top_100_song_lover? 
# cat_top_200_song_lover?
# cat_bottom_20_artist_lover? 
# cat_bottom_50_artist_lover? 
# cat_bottom_100_song_lover? 
# cat_bottom_200_song_lover?

# num_linstened_artist

# num_logs_groupby_artist_max
# num_logs_groupby_artist_max_count
# num_logs_groupby_artist_min
# num_logs_groupby_artist_min_count
# num_logs_groupby_artist_std
# num_logs_groupby_artist_avg
# num_logs_groupby_artist_median

# num_logs
# num_logged_in
# num_cancelled
# num_cancelled_ratio

# cat_gender

# num_itemInSession_max
# num_itemInSession_avg
# num_itemInSession_median
# num_itemInSession_std

# num_length_max
# num_length_min
# num_length_sum
# num_length_std
# num_length_avg
# num_length_median
# num_conti_length_by_page_max
# num_conti_length_by_page_min
# num_conti_length_by_page_std
# num_conti_length_by_ts_max
# num_conti_length_by_ts_min
# num_conti_length_by_ts_std

# num_level_free
# num_level_paid
# num_level_paid_ratio

# num_location_distinct
# cat_max_num_location_state
# cat_max_num_location
# num_max_num_location_geocode_long
# num_max_num_location_geocode_lat

# num_method_get
# num_method_put
# num_method_put_ratio

# (drop page == ‘cancelations’x2)
# num_count_groupby_page x n19 - cancellation x 2 = 17
# num_count_groupby_page_ratio x n19 - - cancellation x 2 = 17
# num_tu_vs_nextsong_ratio
# num_td_vs_nextsong_ratio
# num_plist_vs_nextsong_ratio
# num_roll_vs_nextsong_ratio
# num_ns+tu+td+plist_vs_rest17_ratio
# (More detail on pages….)

# ts_registration
# ts_registration_year
# ts_registration_month
# ts_registration_date
# ts_registration_weekday
# ts_registration_hour
# num_registration-ts_max

# num_distinct_sessionId_count
# num_log_count_groupby_sessionId_max
# num_log_count_groupby_sessionId_min
# num_log_count_groupby_sessionId_mean
# num_log_count_groupby_sessionId_median
# num_log_count_groupby_sessionId_std
# num_sessionId_gap_max
# num_sessionId_gap_min
# num_sessionId_gap_mean
# num_sessionId_gap_median
# num_sessionId_gap_std

# num_total_songs(num_count_groupby_(page=nextSong))
# num_distinct_songs
# num_log_count_groupby_song_max(none-null)
# num_log_count_groupby_song_min(none-null)
# num_log_count_groupby_song_mean(none-null)
# num_log_count_groupby_song_median(none-null)
# num_log_count_groupby_song_std(none-null)

# num_status_200_count
# num_status_307_count
# num_status_404_count
# num_status_200_count_ratio
# num_status_307_count_ratio
# num_status_404_count_ratio

# ts_max-registration
# ts_min-registration
# ts_std
# ts_log_count_by_years
# ts_song_pages_count_by_years
# ts_song_pages_count_by_month / season? / weekday?
# ts_hours_max_all
# ts_hours_min_all
# ts_hours_mean_all
# ts_hours_median_all
# ts_hours_std_all
# ts_hours_max_song_pages
# ts_hours_min_song_pages
# ts_hours_mean_song_pages
# ts_hours_median_song_pages
# ts_hours_std_song_pages

# num_log_count_by_userAgent_device
# num_log_count_by_userAgent_browser_large
# num_log_count_by_userAgent_browser_detailed
# num_page_song_count_by_userAgent_device
# num_page_song_by_userAgent_browser_large
# num_page_song_by_userAgent_browser_detailed
# num_page_util_count_by_userAgent_device
# num_page_util_by_userAgent_browser_large
# num_page_util_by_userAgent_browser_detailed

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.